# Japanese fake news classification

We're going to attempt to classify this dataset. This is a dataset featuring Japanese news articles, from which a part
real, some are half fake, and some are entirely fake.

0: Original article
1: Partially fake
2: Completely fake

Source: [Japanese fakenews dataset](https://www.kaggle.com/tanreinama/japanese-fakenews-dataset)

## Inspection

In [7]:
import re

import pandas as pd
import requests
from sudachipy import tokenizer, dictionary

df = pd.read_csv("fakenews.csv")

df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
isfake,13040.0,1.075613,0.796950,0.0,0.0,1.0,2.00,2.0
nchar_real,13040.0,261.029371,288.257753,0.0,0.0,215.0,407.00,4447.0
nchar_fake,13040.0,328.273160,354.673240,0.0,0.0,255.0,489.25,2582.0


In [8]:
print(f'Rows/Columns: {df.shape}')
print(f"Class distribution is: \n{df['isfake'].value_counts()}\n")
print(df.isnull().sum())

Rows/Columns: (13040, 5)
Class distribution is: 
1    4684
2    4671
0    3685
Name: isfake, dtype: int64

id            0
context       0
isfake        0
nchar_real    0
nchar_fake    0
dtype: int64


From what we can see the dataset does not contain null values, and it's class distribution is mostly even.

Although mostly even, not perfectly even, so while my first thought was accuracy as a metric, i've decided to use the
ROC curve instead.

In [9]:
print(df['context'].head())

0    朝日新聞など各社の報道によれば、宅配便最大手「ヤマト運輸」が日本郵政公社を相手取り、大手コン...
1    11月5日の各社報道によると、諫早湾干拓事業は諫早海人（諫早湾の「海」）に囲まれる大洋に位置...
2    産経新聞、中日新聞によると、2004年から2005年まで、この大会による3年おきの開催を、2...
3    開催地のリオデジャネイロ市に対して、大会期間中のリオデジャネイロオリンピックに関する公式発表...
4    毎日新聞・時事通信によると、2006年2月13日には、グッドウィル・グッゲンハイム・アン・ハ...
Name: context, dtype: object


## Preprocessing

We can mostly follow the standard NLP cleaning methods, albeit with a catch, it's a character based language, so the
traditional packages such as NLTK won't work. That's why we need to find substitutes for the cleaning methods. Let's go
over them:

I've decided to go with [SudachiPy](https://pypi.org/project/SudachiPy/) for tokenization and stemming
[stopwords-ja](https://github.com/stopwords-iso/stopwords-ja/blob/master/stopwords-ja.json) (link to json file) for stopword removal.

I decided to not use lemmatization, since there are a lot of words that might _seem_ similar, but are vastly different
taking into account their context.

### Removing features

Everything besides the actual text and the label class is just noise, so we'll take those out.

In [10]:
labels = df.pop('isfake')
data = df.pop('context')

### Tokenization

We're going to tokenize our sentences. This we'll do with Sudachi, a morphological analyzer. The short version of
what that means is that we're splitting our sentences up into pieces, aka tokens. We need a specialized analyzer for
this task, because you can't just randomly split it up by character, because then you'd lose context. After tokenization
is complete we can do the rest of the preprocessing steps, which we usually apply individually per character.

### Special characters

We need to also remove some special characters that will be of no use to the classifier, think of stuff such as commas
and question marks, etc. We do this via a regex filter. I spent some time looking for the exactly right one online,
since I wanted to keep all japanese characters, but remove junk such as symbols. Speaking of symbols for some reason
the Japanese have their own version of them for some reason (。、？  notice how they're different?), which is pretty
tricky. While you'd want to remove irrelevant characters, you also don't want to remove all symbols, since some are
very important to the language (e.g. ー, which lengthens vowels.).

The conclusion here is that you'll have to build your own japanese regex filter to suit your exact needs. Definitely
check out [this gist](https://gist.github.com/terrancesnyder/1345094) for that.

Check out the pre-processing function below for the specific regex i used.

### Stemming

Stemming implies reducing the form of a word to its stem. E.g. 食べている -->　食べる, した　--> する. This is useful to
reduce the amount of variants of a word which mostly mean the exact same thing (from the perspective of the model)

### Stopwords

Stopword removal implies the removal of certain words which appear in an excessive frequency in the language. Common
examples for English are "the", "and" and so forth. These are useful to remove because they lose meaning viewed
individually.

In [11]:
def sample(data, limit=3):
    """
    Quick util method to display samples of the sentences
    :param data: collection
    :param limit: how much samples to show
    """
    for x in range(0, limit, 1):
        print(f"{data[x]}\n")


sample(data)

朝日新聞など各社の報道によれば、宅配便最大手「ヤマト運輸」が日本郵政公社を相手取り、大手コンビニエンスストア「ローソン」でのサービス提供の差し止めなどを求めていた訴訟で、2006年1月19日、東京地方裁判所でヤマト運輸の請求を棄却する判決が下された。2004年のローソンでの郵便小包サービス「ゆうパック」の受付業務開始に際し、ヤマト運輸は「独占禁止法に違反する不当な廉売」として、日本郵政公社を相手取り、サービス提供の差し止めなどを求めていた。朝日新聞によれば、提訴の内容は、2004年11月のローソンでの「ゆうパック」の受付サービス提供の開始に関連し、租税などの優遇措置を受けている日本郵政公社が、配送料金（運賃）などの有利な取引条件でローソンで「ゆうパック」を開始させたのは、独占禁止法の不当廉売に当たり、ヤマト運輸の利益を侵害されるとして、「ゆうパック」サービス提供の差し止めなどを求めていたもの。朝日新聞によれば、判決内容はヤマト運輸の主張を全面的に否定しており、今後の「ゆうパック」サービスの拡大に弾みがつくものと考えられる。日本郵政公社は、公正妥当な判決とのコメントを出した。一方、ヤマト運輸は、高等裁判所への控訴など、今後の対応については検討するとアナウンスしている。

11月5日の各社報道によると、諫早湾干拓事業は諫早海人（諫早湾の「海」）に囲まれる大洋に位置することから、人身売買により、環境問題に加え、環境保護にも関心が向けられた。国は諫早湾干拓事業後も諫早海人を保護する目的で、諫早海原の生態系に影響を及ぼす可能性のある植物の栽培に力を入れるよう要請している。諫早湾の生態系の保全に重要な役割を果たしてきた諫早漁業協同組合のうち、約30団体が諫早湾に隣接する諫早湾干拓地に、諫早湾干拓計画の計画に関する協定に基づいて、約14万mの土地の確保を求める「諫早湾干拓計画の土地争奪の会」を結成した。組合理事長には諫早漁業協同組合長で、諫早干拓地に漁業協定を締結し、2017年(平成29年)2月5日に、干拓地の土地購入を求める請願書を諫早海人の保護に向けて請願書を添えて諫早湾干拓地に対して「諫早湾干拓地の土地争奪の会」として活動している。

産経新聞、中日新聞によると、2004年から2005年まで、この大会による3年おきの開催を、2006年から2年連続で実施したことがある。また、

In [12]:
def pre_process_text(text, stopwords, tokenizer_obj):
    """
    Cleans the text of unnecessary features
    :param text: sentence to clean
    :param stopwords: list of very commonly used words
    :param tokenizer_obj: SudachiPy tokenizer object
    :return: cleaned string
    """
    # One by one: (kanji), (hiragana and katakana), (western alphabet and numbers), (western alphabet and numbers
    # except it's the off-looking japanese version), and unicode flag
    pattern = re.compile(r'([一-龯]+)|([ぁ-んァ-ン]+)|([a-zA-Z0-9]+)|([ａ-ｚＡ-Ｚ０-９]+)|[ー+]', re.UNICODE)
    text = ''.join([x.group() for x in re.finditer(pattern, text.lower().strip())])

    lst_text = [m.dictionary_form() for m in tokenizer_obj.tokenize(text, mode)]

    # Remove Stopwords
    lst_text = [word for word in lst_text if word not in stopwords]

    # Rejoin tokenized string
    text = " ".join(lst_text)
    return text


tokenizer_obj = dictionary.Dictionary().create()
mode = tokenizer.Tokenizer.SplitMode.C
lst_stopwords = requests.get(
    'https://raw.githubusercontent.com/stopwords-iso/stopwords-ja/master/stopwords-ja.json').json()

print("Starting cleaning phase, this may take a few minutes...")
data = [pre_process_text(i, lst_stopwords, tokenizer_obj) for i in data]
sample(data, 5)

Starting cleaning phase, this may take a few minutes...
朝日新聞 各社 報道 よる 宅配便 最大手 ヤマト 運輸 日本郵政 公社 相手 取る 大手 コンビニエンスストアローソン サービス 提供 差し止め 求める 訴訟 2006 年 1 月 19 日 東京 地方裁判所 ヤマト 運輸 請求 棄却 判決 下す 2004 年 ローソン 郵便 小包 サービス ゆう パック 受付 業務 開始 際する ヤマト 運輸 独占禁止法 違反 不当 廉売 日本郵政 公社 相手 取る サービス 提供 差し止め 求める 朝日新聞 よる 提訴 内容 2004 年 11 月 ローソン ゆう パック 受付 サービス 提供 開始 関連 租税 優遇措置 受ける 日本郵政 公社 配送 料金 運賃 有利 取引 条件 ローソン ゆう パック 開始 独占禁止法 不当廉売 当たる ヤマト 運輸 利益 侵害 ゆう パック サービス 提供 差し止め 求める 朝日新聞 よる 判決 内容 ヤマト 運輸 主張 全面的 否定 おる 今後 ゆう パック サービス 拡大 弾み つく 考える 日本郵政 公社 公正 妥当 判決 コメント 出す 一方 ヤマト 運輸 高等裁判所 控訴 今後 対応 つく 検討 アナウンス

11 月 5 日 各社 報道 よる 諫早湾 干拓 事業 諫早 海人 諫早湾 海 囲む 大洋 位置 人身 売買 よる 環境 問題 加える 環境 保護 関心 向ける 国 諫早湾 干拓 事業 後 諫早 海人 保護 目的 諫早 海原 生態系 影響 及ぼす 可能性 植物 栽培 力 入れる 要請 諫早湾 生態系 保全 重要 役割 果たす くる 諫早 漁業協同組合 約 30 団体 諫早湾 隣接 諫早湾 干拓地 諫早湾 干拓 計画 計画 関する 協定 基づく 約 14万 m 土地 確保 求める 諫早湾 干拓 計画 土地 争奪 会 結成 組合 理事長 諫早 漁業協同組合 長 諫早 干拓地 漁業 協定 締結 2017 年 平成 29 年 2 月 5 日 干拓地 土地 購入 求める 請願書 諫早 海人 保護 向ける 請願書 添える 諫早湾 干拓地 対する 諫早湾 干拓地 土地 争奪 会 活動

産経 新聞 中日 新聞 よる 2004 年 2005 年 大会 よる 3 年 おき 開催 2

## NLP technique

Now we need to choose a technique to actually process this text, because after all, our neural network only accepts
numbers, not japanese characters. Some examples of the current most popular techniques are Bag of Words, TF_IDF Scheme,
BERT, word2vec.

I've decided to use word2vec here, which maps words and/or phrases to vectors. word2vec comes with 2 options, Skip Gram
and Continuous Bag of Words, which are essentially mirrored versions of each other. CBOW is trained to predict a single
word from multiple context words, SG is trained to predict multiple words from a single context word.

## Conclusion

Package wise, we still have a long way to go. The Japanese equivalents of the NLP packages are more underveloped,
less reliable, and generally slower too. This can be mostly attributed to the language being difficult, but it's
something you have to keep in mind while working with Japanese text. Maybe in a few years the tools will be at a good
level.


## References

[Preprocessing Methods and Tools in Modelling Japanese for Text Classification](https://www.researchgate.net/publication/335337209_Preprocessing_Methods_and_Tools_in_Modelling_Japanese_for_Text_Classification)
Paper detailing tools for Japanese NLP analysis. A few years old, but still relevant as of current date.

[nlp-recipes-ja](https://github.com/upura/nlp-recipes-ja)
Github repository containing a ton of samples for Japanese text analysis in Python.
